In [ ]:
import tensorflow as tf
from sklearn import datasets

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np

In [ ]:
iris = datasets.load_iris()
features = iris['data']
target = iris['target']
N = len(target)
shuffle_index = np.arange(len(target))
np.random.shuffle(shuffle_index)
features = features[shuffle_index, :]
target = target[shuffle_index]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(features[:,1], features[:, 2], features[:, 3], c=target)
ax.view_init(30, 210)
plt.show()

In [ ]:
TRAIN_SIZE = 120
train_x, test_x = features[:TRAIN_SIZE], features[TRAIN_SIZE:]
train_y, test_y = target[:TRAIN_SIZE], target[TRAIN_SIZE:]

In [ ]:
slices = tf.data.Dataset.from_tensor_slices(({'feat0': train_x[:,0],
                                              'feat1': train_x[:, 1],
                                              'feat2': train_x[:, 2],
                                              'feat3': train_x[:, 2]},
                                             train_y))

In [ ]:
print(slices)

In [ ]:
def train_input_fn(features, labels, batch_size=128):
    # Can do all features as one:
    # dataset = tf.data.Dataset.from_tensor_slices(({'feats': features}, labels))
    # Or separate "columns":
    dataset = tf.data.Dataset.from_tensor_slices(({'feat0': features[:,0],
                                                   'feat1': features[:, 1],
                                                   'feat2': features[:, 2],
                                                   'feat3': features[:, 2]},
                                                   labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset

In [ ]:
def eval_input_fn(features, labels, batch_size=32):
    feats = {'feat0': features[:,0], 'feat1': features[:, 1],
             'feat2': features[:, 2],'feat3': features[:, 2]}
    
    if labels is None:
        inputs = feats
    else:
        inputs = (feats, labels)

    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
train_input_fn(train_x, train_y)

In [ ]:
my_feature_columns = []
for key in ['feat0', 'feat1', 'feat2', 'feat3']:
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [ ]:
# More on feature column:
# https://www.tensorflow.org/guide/feature_columns

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],
    n_classes=3)

In [ ]:
classifier.train(input_fn=lambda: train_input_fn(train_x, train_y, batch_size=100), steps=1000)

In [ ]:
eval_result = classifier.evaluate(input_fn=lambda: eval_input_fn(test_x, test_y, batch_size=100))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [ ]:
pred_results = classifier.predict(input_fn=lambda: eval_input_fn(test_x[:5], test_y[:5], batch_size=100))

In [ ]:
for pred, expect in zip(pred_results, test_y[:5]):
    class_id = pred['class_ids'][0]
    prob = pred['probabilities'][class_id]
    print("Expected: {} Pred: {} Prob: {}".format(expect, class_id, prob))